In [1]:
import pathlib
import zipfile
import requests

BASE_DIR = pathlib.Path().resolve().parent
DATASET_DIR = BASE_DIR / "datasets"
EXPORT_DIR = DATASET_DIR / "exports"
EXPORT_DIR.mkdir(exist_ok=True, parents=True)
OTTER_DATASET_PATH = EXPORT_DIR / "otterDUMMY-dataset.csv"

ZIPS_DIR = DATASET_DIR / "zips"
ZIPS_DIR.mkdir(exist_ok=True, parents=True)

OTTER1_ZIP_PATH = ZIPS_DIR / "otter1DUMMY-dataset.zip"
OTTER2_ZIP_PATH = ZIPS_DIR / "otter2DUMMY-dataset.zip"

In [2]:
OTTER1_ZIP = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip" 
OTTER2_ZIP = "https://archive.ics.uci.edu/static/public/380/youtube+spam+collection.zip"

In [3]:
!curl $OTTER1_ZIP -o $OTTER1_ZIP_PATH
!curl $OTTER2_ZIP -o $OTTER2_ZIP_PATH

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 16309    0 16309    0     0  20400      0 --:--:-- --:--:-- --:--:-- 20437
100  198k    0  198k    0     0   104k      0 --:--:--  0:00:01 --:--:--  104k
100  198k    0  198k    0     0   104k      0 --:--:--  0:00:01 --:--:--  104k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 57269    0 57269    0     0  61634      0 --:--:-- --:--:-- --:--:-- 61712
100  159k    0  159k    0     0  99105      0 --:--:--  0:00:01 --:--:-- 99191


In [4]:
response1 = requests.get(OTTER1_ZIP)
with open(OTTER1_ZIP_PATH, 'wb') as f:
    f.write(response1.content)

response2 = requests.get(OTTER2_ZIP)
with open(OTTER2_ZIP_PATH, 'wb') as f:
    f.write(response2.content)

INPUT_CLASSIFIER_DIR = DATASET_DIR / 'input-classifier'
OTTER1_DIR = INPUT_CLASSIFIER_DIR / 'otter1'
OTTER2_DIR = INPUT_CLASSIFIER_DIR / 'otter2'

OTTER1_DIR.mkdir(exist_ok=True, parents=True)
OTTER2_DIR.mkdir(exist_ok=True, parents=True)

In [5]:
with zipfile.ZipFile(OTTER1_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(OTTER1_DIR)

with zipfile.ZipFile(OTTER2_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(OTTER2_DIR)


In [6]:
otter1_input_path = OTTER1_DIR / "SMSSpamCollection" # tsv
# otter1_input_path.read_text()

In [7]:
for path in OTTER2_DIR.glob("*"):
    print(path)

C:\Users\syxrw\Documents\TeamProjects\iGEM\criGEM-NUS2023\otter-backend\otter-api\datasets\input-classifier\otter2\Youtube01-Psy.csv
C:\Users\syxrw\Documents\TeamProjects\iGEM\criGEM-NUS2023\otter-backend\otter-api\datasets\input-classifier\otter2\Youtube02-KatyPerry.csv
C:\Users\syxrw\Documents\TeamProjects\iGEM\criGEM-NUS2023\otter-backend\otter-api\datasets\input-classifier\otter2\Youtube03-LMFAO.csv
C:\Users\syxrw\Documents\TeamProjects\iGEM\criGEM-NUS2023\otter-backend\otter-api\datasets\input-classifier\otter2\Youtube04-Eminem.csv
C:\Users\syxrw\Documents\TeamProjects\iGEM\criGEM-NUS2023\otter-backend\otter-api\datasets\input-classifier\otter2\Youtube05-Shakira.csv
C:\Users\syxrw\Documents\TeamProjects\iGEM\criGEM-NUS2023\otter-backend\otter-api\datasets\input-classifier\otter2\__MACOSX


In [8]:
import pandas as pd

In [9]:
otter1_input_path = OTTER1_DIR / "SMSSpamCollection"
otter1_df = pd.read_csv(otter1_input_path, sep='\t', header=None)
otter1_df.columns = ['label', 'text']
otter1_df['source'] = 'otter-1'
otter1_df.tail()

,label,text,source
5567,spam,This is the 2nd time we have tried 2 contact u...,otter-1
5568,ham,Will ü b going to esplanade fr home?,otter-1
5569,ham,"Pity, * was in mood for that. So...any other s...",otter-1
5570,ham,The guy did some bitching but I acted like i'd...,otter-1
5571,ham,Rofl. Its true to its name,otter-1


In [10]:
1 == True

True

In [12]:
otter2_dfs = []
for path in OTTER2_DIR.glob("*"):
    raw_df = pd.read_csv(path)
    raw_df.rename(columns={"CLASS": 'raw_label',"CONTENT": "text"}, inplace=True)
    raw_df['label'] = raw_df['raw_label'].apply(lambda x: "spam" if str(x) == "1" else "ham")
    raw_df['raw_source'] = str(path.name)
    raw_df['source'] = 'otter-2'
    df = raw_df.copy()[['label', 'text', 'source']]
    otter2_dfs.append(df)
    # print(df.head())

otter2_df = pd.concat(otter2_dfs)

In [13]:
otter2_df.head()

,label,text,source
0,spam,"Huh, anyway check out this you[tube] channel: ...",otter-2
1,spam,Hey guys check out my new channel and our firs...,otter-2
2,spam,just for test I have to say murdev.com,otter-2
3,spam,me shaking my sexy ass on my channel enjoy ^_^ ﻿,otter-2
4,spam,watch?v=vtaRGgvGtWQ Check this out .﻿,otter-2


In [14]:
df = pd.concat([otter1_df, otter2_df])
df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",otter-1
1,ham,Ok lar... Joking wif u oni...,otter-1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,otter-1
3,ham,U dun say so early hor... U c already then say...,otter-1
4,ham,"Nah I don't think he goes to usf, he lives aro...",otter-1


In [ ]:
df.to_csv(OTTER_DATASET_PATH, index=False)